In [21]:
# Connect to a specific database
cluster_name = "echo"
database_name = "sample_mflix"
collection_name = "movies"

In [22]:
# Init
import json
import os

import requests
from IPython.core.display import Markdown
# https://github.com/anthropics/anthropic-sdk-python
from anthropic import Anthropic
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient

# Load env variables for MongoDB Atlas CLI, MongoDB database client and Anthropic client
load_dotenv()
uri = os.getenv("MONGO_URI")

# Create MongoBD Client, point to a specific database/collection
mongodb_client = MongoClient(uri)
namespace = database_name + "." + collection_name
db = mongodb_client[database_name][collection_name]

# Create Anthropic client
anthropic_client = Anthropic()
base_context = []

## MongoDB Context

In [23]:
docs = [
    "https://www.mongodb.com/docs/manual/tutorial/equality-sort-range-rule/",
    "https://www.mongodb.com/docs/manual/reference/explain-results/",
    "https://www.mongodb.com/docs/manual/tutorial/analyze-query-plan/",
    "https://www.mongodb.com/docs/manual/tutorial/optimize-query-performance-with-indexes-and-projections/",
    "https://www.mongodb.com/docs/atlas/performance-advisor/",
    "https://www.mongodb.com/docs/atlas/performance-advisor/index-ranking/",
    "https://www.mongodb.com/docs/manual/reference/operator/aggregation/indexStats/",
    "https://www.mongodb.com/docs/cloud-manager/reference/api/performance-advisor/get-suggested-indexes/"
]

In [24]:
# Add to context
def webpage_as_context(url):
    response = requests.get(url)

    soup = BeautifulSoup(response.text, 'html.parser')
    body = soup.select('div.body')[0]
    text = body.get_text(separator=" ", strip=True)

    return {
        "role": "user",
        "content": f"MongoDB documentation from {url}\n{text}",
    }


base_context = []
for url in docs:
    context_message = webpage_as_context(url)
    base_context.append(context_message)


## Indexes

In [25]:
designed_indexes = [
    {'_id': 1},
    {'_fts': 'text', '_ftsx': 1},
    {'genres': 1},
    {'text': 1},
    {'year': 1},
    {'type': 1, 'imdb.rating': -1, 'year': 1}
]

In [39]:
# Organize indexes

# Designed indexes
designed_indexes = [str(index_definition) for index_definition in designed_indexes]

# Current indexes in database
# https://www.mongodb.com/docs/manual/reference/operator/aggregation/indexStats/
current_indexes = []
index_stats = list(db.aggregate([{"$indexStats": {}}]))

for idx in index_stats:
    current_indexes.append(str(idx["key"]))

# Suggested indexes
# API details: https://www.mongodb.com/docs/cloud-manager/reference/api/performance-advisor/get-suggested-indexes/
suggested_indexes = set()
suggested_indexes_details = []

# CLI https://www.mongodb.com/docs/atlas/cli/current/command/atlas-clusters-list/
clusters_raw = !atlas clusters list -o json
clusters = json.loads(clusters_raw.s)["results"]
cluster = None

for c in clusters:
    if cluster_name == c["name"]:
        cluster = c
        break
cluster_hosts = cluster["connectionStrings"]["standard"].split("/")[2].split(",")

# CLI https://www.mongodb.com/docs/atlas/cli/current/command/atlas-performanceAdvisor-suggestedIndexes/
for host in cluster_hosts:
    index_suggestions_raw = !atlas performanceAdvisor suggestedIndexes list --processName {host} -o json
    index_suggestions = json.loads(index_suggestions_raw.s)

    for suggestion in index_suggestions["suggestedIndexes"]:
        if namespace not in suggestion["namespace"]: continue

        index_definition = {}
        for record in suggestion["index"]:
            index_definition.update(record)

        if str(index_definition) in suggested_indexes:
            continue

        suggested_indexes.add(str(index_definition))

        sample_queries = []
        for sample_query in index_suggestions["shapes"]:
            if sample_query["id"] in suggestion["impact"]:
                del sample_query["operations"][0]["stats"]["ts"]
                suggested_indexes_details.append({
                    "index_definition": index_definition,
                    "sample_query": json.dumps(sample_query["operations"][0]["predicates"]),
                    "query_count": sample_query["count"],
                    "stats": json.dumps(sample_query["operations"][0]["stats"])
                })
                # Extract a single query example in index recommendation
                break

# Compose index summary
message = "Designed indexes:\n"

for idx in designed_indexes:
    status = "\033[91mx\033[0m "
    if idx in current_indexes:
        status = "\033[92m√\033[0m "
    if idx in suggested_indexes:
        status = "\033[91m+\033[0m "

    message += status + str(idx)
    message += "\n"

message += "\n"
message += "Current indexes:\n"

for idx in current_indexes:
    status = "\033[93m?\033[0m "
    if idx in designed_indexes:
        status = "\033[92m√\033[0m "
    if idx in suggested_indexes:
        status = "\033[92m+\033[0m "

    message += status + str(idx) + "\n"

message += "\n"
message += "Suggested indexes:\n"
for si in suggested_indexes_details:
    message += "\033[92m+\033[0m " + str(si["index_definition"]) + "\n"
    message += "    query: " + str(si["query_count"]) + " x " + str(si["sample_query"]) + "\n"
    message += "    stats: " + str(si["stats"]) + "\n\n"

print(message)

Designed indexes:
√ {'_id': 1}
√ {'_fts': 'text', '_ftsx': 1}
√ {'genres': 1}
x {'text': 1}
√ {'year': 1}
+ {'type': 1, 'imdb.rating': -1, 'year': 1}

Current indexes:
√ {'_id': 1}
√ {'genres': 1}
√ {'year': 1}
√ {'_fts': 'text', '_ftsx': 1}
? {'genres': 1, 'poster': 1}

Suggested indexes:
+ {'type': 1, 'imdb.rating': -1, 'year': 1}
    query: 16 x [{"aggregate": [{"$match": {"type": "movie", "year": {"$in": [{"$numberInt": "2010"}, {"$numberInt": "2011"}, {"$numberInt": "2012"}, {"$numberInt": "2013"}, {"$numberInt": "2014"}, {"$numberInt": "2015"}]}}}, {"$sort": {"imdb.rating": {"$numberInt": "-1"}}}, {"$limit": {"$numberInt": "10000"}}]}]
    stats: {"ms": 108, "nReturned": 101, "nScanned": 26861}




## Queries

#### 1. Find movies by type, year, sort by imdb.rating
Index `{'type': 1, 'imdb.rating': -1, 'year': 1}`

In [35]:
pipeline = [
    {"$match": {
        "type": "movie",
        "year": {"$in": [2010, 2011, 2012, 2013, 2014, 2015]}
    }},
    {"$sort": {"imdb.rating": -1}},
    {"$limit": 10000}
]

#### 2. Find movies with missing poster by genre

Index `{'genres': 1}`



In [45]:
pipeline = [
    {"$match": {
        "genres": {"$in": ["Animation", "Short"]},
        "poster": {"$exists": "false"}
    }},
    {"$project": {"_id": 1, "title": 1}}
]

#### 3. Aggregation with an $or operator
Indexes: `{'year': 1}`, `{'genres': 1}`

In [29]:
pipeline = [
    {"$match": {
        "$or": [{"year": 2000}, {"genres": "Animation"}]
    }},
    {"$project": {"_id": 1, "title": 1}}
]

## Query explain

In [46]:
# Explain query
# https://www.mongodb.com/docs/manual/reference/explain-results/
explain = mongodb_client.get_database(database_name).command(
    'explain',
    {
        'aggregate': collection_name,
        'pipeline': pipeline,
        'cursor': {}
    },
    verbosity='allPlansExecution'
)

context = base_context.copy()
context.extend([
    {
        "role": "user",
        "content": f"Format a json explain plan using a template. Use markdown format for output. Do not provide text summary in the end ",
    },
    {
        "role": "user",
        "content": """Template:
### Query Overview
```
nReturned: 7345
executionTimeMillis: 16
totalKeysExamined: 7465
totalDocsExamined: 7345
```

### Indexes Used
```
- genres_1: {genres: 1}
- year_1: {year: 1}
```

### Execution Stages
```
SUBPLAN
└── PROJECTION_SIMPLE
    └── FETCH
        └── OR
            ├── IXSCAN (genres_1)
            │   nReturned: 4560
            │   keysExamined: 4560
            │
            └── IXSCAN (year_1)
                nReturned: 2905
                keysExamined: 2905
```""",
    },
    {
        "role": "user",
        "content": f"Query explain: {explain}",
    }
])
response = anthropic_client.messages.create(
    max_tokens=1024,
    messages=context,
    model="claude-3-5-sonnet-latest",
)

display(Markdown(response.content[0].text))

### Query Overview
```
nReturned: 4405
executionTimeMillis: 15
totalKeysExamined: 6772
totalDocsExamined: 5715
```

### Indexes Used
```
- genres_1: {genres: 1}
- genres_1_poster_1: {genres: 1, poster: 1} (rejected)
```

### Execution Stages
```
PROJECTION_SIMPLE (transformBy: {_id: true, title: true})
└── FETCH (filter: {poster: {$exists: true}})
    └── IXSCAN (genres_1)
        nReturned: 5715
        keysExamined: 6772
        indexBounds: {genres: ["Animation", "Short"]}
        isMultiKey: true
        seeks: 2
        dupsTested: 6770
        dupsDropped: 1055
```

In [47]:
# Measure end-to-end execution time
time = %timeit -o list(db.aggregate(pipeline=pipeline))

375 ms ± 90.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
# Generate summary
context = base_context.copy()
context.extend([
    {
        "role": "user",
        "content": f"Act as a MongoDB advisor. You will be provided with information on indexes, example query that is being evaluated and a query explain plan. Provide a brief summary of explain plan, recommendations for query performance and indexes. Use markdown format for output, escape $ $ character sequence in markdown if it's not in the code block.",
    },
    {
        "role": "user",
        "content": f"Current indexes: {current_indexes}, designed indexes: {designed_indexes}, suggested indexes: {suggested_indexes_details}",
    },
    {
        "role": "user",
        "content": f"Index stats: {index_stats}",
    },
    {
        "role": "user",
        "content": f"Query: {pipeline}",
    },
    {
        "role": "user",
        "content": f"Query explain: {explain}",
    },
    {
        "role": "user",
        "content": f"Execution time on on the client side: {time}",
    }
])
response = anthropic_client.messages.create(
    max_tokens=1024,
    messages=context,
    model="claude-3-5-sonnet-latest",
)

display(Markdown(response.content[0].text))
context.extend([
    {
        "role": "assistant",
        "content": response.content[0].text,
    }
])

# Query Analysis and Recommendations

## Explain Plan Summary
- Query uses an index scan on `genres_1` index
- Returns 4,405 documents
- Examines 6,772 index keys and 5,715 documents
- Execution time: 15ms on server side, 375ms on client side
- Query includes a projection stage for `_id` and `title` fields
- Uses multikey index due to `genres` being an array field

## Performance Analysis
1. **Index Usage**: 
   - Query effectively uses the `genres_1` index for the `$in` condition
   - However, the `poster` existence check requires a document scan after index lookup
   - Query examined more documents (5,715) than returned (4,405) due to filtering

2. **Current Index Stats**:
   - `genres_1` index is frequently used (144 ops)
   - `genres_1_poster_1` compound index exists but wasn't chosen as winning plan

## Recommendations

1. **Index Optimization**:
   ```javascript
   { "genres": 1, "poster": 1 }  // Already exists but could be more effective
   ```
   Consider creating a sparse or partial index if many documents don't have `poster` field.

2. **Query Optimization**:
   - Consider changing `poster: {$exists: 'false'}` to `poster: null` if appropriate for your data model
   - If possible, batch process results to reduce client-side execution time

3. **Monitoring**:
   - The text index (`cast_text_fullplot_text_genres_text_title_text`) shows 0 ops - consider removing if unused
   - The `genres_1_poster_1` index shows 0 ops - evaluate if it's properly designed for your queries

4. **Query Shape Optimization**:
   - Consider adding a limit if you don't need all 4,405 documents
   - Use projection as currently done to minimize data transfer

The significant difference between server execution time (15ms) and client execution time (375ms) suggests network transfer or client-side processing might be the bottleneck rather than index efficiency.

In [49]:
follow_up_question = "Why some indexes were considered but rejected for this query?"

In [50]:
# Chat
context.extend([
    {
        "role": "user",
        "content": follow_up_question,
    }
])
response = anthropic_client.messages.create(
    max_tokens=1024,
    messages=context,
    model="claude-3-5-sonnet-latest",
)

display(Markdown(response.content[0].text))

context.extend([
    {
        "role": "assistant",
        "content": response.content[0].text,
    }
])

# Rejected Plan Analysis

Let's analyze why the query planner rejected alternative plans, specifically looking at the `rejectedPlans` section from the explain output.

## Rejected Plan Details
```javascript
{
    'stage': 'PROJECTION_SIMPLE',
    'inputStage': {
        'stage': 'FETCH',
        'filter': {'poster': {'$exists': True}},
        'inputStage': {
            'stage': 'IXSCAN',
            'keyPattern': {'genres': 1, 'poster': 1},
            'indexName': 'genres_1_poster_1'
        }
    }
}
```

## Rejection Reasons

1. **Performance Trial Results**:
   - Winning plan score: 1.834910743801653
   - Rejected plan score: 1.0002
   - The query planner chose the simpler `genres_1` index because it performed better during the trial period

2. **Trial Statistics Comparison**:
   ```
   Winning Plan (genres_1):
   - nReturned: 101
   - totalKeysExamined: 121
   - totalDocsExamined: 121
   
   Rejected Plan (genres_1_poster_1):
   - nReturned: 0
   - totalKeysExamined: 121
   - totalDocsExamined: 121
   ```

3. **Key Reasons for Rejection**:
   - The compound index `genres_1_poster_1` didn't provide better selectivity
   - The `$exists` condition on `poster` field couldn't effectively use the compound index
   - The rejected plan returned 0 documents during trial period while the winning plan returned 101 documents

## Understanding Index Selection
The MongoDB query planner:
1. Generates candidate plans for each viable index
2. Runs each plan on a subset of data
3. Scores plans based on execution metrics
4. Caches the winning plan for similar query shapes

In this case, even though `genres_1_poster_1` includes both fields in the query, it wasn't more efficient because:
- The `$exists` operator doesn't benefit from index ordering
- The additional index key in the compound index added overhead without improving selectivity
- The simpler single-field index provided sufficient filtering with less overhead